In [ ]:
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

import warnings
import os

warnings.filterwarnings("ignore")
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
os.environ["OMP_NUM_THREADS"] = "8"

# Cell segmentation

In [ ]:
import lazyslide as zs

In [ ]:
wsi = zs.datasets.sample(with_data=False)

In [ ]:
wsi

In [ ]:
zs.pp.find_tissues(wsi)
zs.pp.tile_tissues(wsi, 512, background_fraction=0.95)

In [ ]:
zs.pl.tiles(wsi, linewidth=0.4)

In [ ]:
zs.seg.cells(wsi, num_workers=0, device="mps", batch_size=4)

In [ ]:
v = zs.pl.WSIViewer(wsi)
v.add_image()
v.add_polygons("cells")
v.add_zoom(1000, 1500, 2100, 2600)
v.show()

Run on a bigger slide

In [ ]:
from wsidata import open_wsi
import pooch

slide = pooch.retrieve(
    url="https://brd.nci.nih.gov/brd/imagedownload/GTEX-11DXX-1626",
    fname="GTEX-11DXX-1626.svs",
    path=".",
    known_hash="sha256:b88f44637fdbb80f545642ee4ea7ea3d155bea533a9052c47c01580f55bd206c",
)
wsi = open_wsi(slide)

In [ ]:
wsi

In [ ]:
zs.pp.find_tissues(wsi)
zs.pp.tile_tissues(wsi, 512, overlap=0.1, background_fraction=0.3)

In [ ]:
zs.seg.cells(wsi, device="mps")

In [ ]:
zs.seg.nulite(wsi, num_workers=0, device="mps")

In [ ]:
wsi["cell_types"] = wsi["cell_types"][wsi["cell_types"]["prob"] > 0.75]

In [ ]:
v = zs.pl.WSIViewer(wsi)
v.add_image()
v.add_polygons("cell_types", color_by="name")
v.add_zoom(1000, 1500, 2100, 2600)
v.show()

In [ ]:
v = zs.pl.WSIViewer(wsi)
v.add_image()
v.set_tissue_id(3)
v.add_polygons("cells")
# v.add_zoom(0., 1500, 2100, 2600)
# v.show()

In [ ]:
v.set_viewport(43500, 12000, 1000, 1000)

In [ ]:
v.show()

In [ ]:
wsi["cells"].to_file(f"{wsi.name}_cells.geojson")

In [ ]:
zs.io.export_annotations(wsi, "cells", in_bounds=True, file=f"{wsi.name}_cells.geojson")